In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import shap
import os
from datetime import datetime
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support
from xgboost import XGBClassifier
import xgboost as xgb
import joblib
from sklearn.metrics import fbeta_score, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import os

In [2]:
data_A  = pd.read_csv("F:/New_Drugs/Li_Endo/Li_Endo_Training.csv")  # Ganti dengan lokasi dataset A
data_A1 = data_A.drop(columns=['Sample_ID', 'Drug_Name', 'risk_level', 'Vm_Peak', 'Ca_Peak','Ca_Diastole','Max_dVm/dt','Catri','CaD50'])
print(data_A1)

       dVm/dt_repol  Vm_Resting       APD90       APD50  APDtri       CaD90  \
0         -0.300345  -88.002062  391.600435  318.450435  73.150  688.500001   
1         -0.317049  -88.003051  370.700439  303.150439  67.550  682.500001   
2         -0.279408  -87.999663  420.275299  340.875299  79.400  686.000001   
3         -0.255510  -88.001840  455.900506  365.575506  90.325  709.500001   
4         -0.298442  -88.001574  394.375319  321.325319  73.050  684.500001   
...             ...         ...         ...         ...     ...         ...   
21995     -0.298215  -88.039405  350.900470  281.325470  69.575  817.500001   
21996     -0.283611  -88.036115  371.725455  297.850455  73.875  815.500001   
21997     -0.290358  -88.034500  366.000421  293.750421  72.250  806.000001   
21998     -0.281967  -88.034736  374.300405  300.075404  74.225  810.500001   
21999     -0.280863  -88.034116  376.475531  301.800531  74.675  809.000001   

           qNet   qInward  risk_code  
0      0.069

In [3]:
X_data_bfr = data_A1.drop(['risk_code'], axis=1)
y_data_label = data_A1['risk_code']
print(X_data_bfr)
print(y_data_label)

       dVm/dt_repol  Vm_Resting       APD90       APD50  APDtri       CaD90  \
0         -0.300345  -88.002062  391.600435  318.450435  73.150  688.500001   
1         -0.317049  -88.003051  370.700439  303.150439  67.550  682.500001   
2         -0.279408  -87.999663  420.275299  340.875299  79.400  686.000001   
3         -0.255510  -88.001840  455.900506  365.575506  90.325  709.500001   
4         -0.298442  -88.001574  394.375319  321.325319  73.050  684.500001   
...             ...         ...         ...         ...     ...         ...   
21995     -0.298215  -88.039405  350.900470  281.325470  69.575  817.500001   
21996     -0.283611  -88.036115  371.725455  297.850455  73.875  815.500001   
21997     -0.290358  -88.034500  366.000421  293.750421  72.250  806.000001   
21998     -0.281967  -88.034736  374.300405  300.075404  74.225  810.500001   
21999     -0.280863  -88.034116  376.475531  301.800531  74.675  809.000001   

           qNet   qInward  
0      0.069732  0.9926

In [ ]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define parameter grid for Grid Search
param_grid = {
    'n_estimators': [300],
    'max_depth': [None],
    'min_samples_split': [5],
    'min_samples_leaf': [1]
}

# Create a GridSearchCV instance
grid = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=skf, n_jobs=-1)

# Perform Grid Search
grid.fit(X_data_bfr, y_data_label)

# Get the best hyperparameters
best_params = grid.best_params_

# Create a folder name based on the best parameters
folder_name = f"E:/backup_CML_1/New_Drugs/Li_Endo/Result_Revision_2/RF_6_feature_nestimators_{best_params['n_estimators']}_maxdepth_{best_params['max_depth']}_minsplit_{best_params['min_samples_split']}_minleaf_{best_params['min_samples_leaf']}"
os.makedirs(folder_name, exist_ok=True)

# Train the best model using 10-fold cross-validation and save models
fold = 0

for train_index, test_index in skf.split(X_data_bfr, y_data_label):
    fold += 1
    X_train, X_test = X_data_bfr.iloc[train_index], X_data_bfr.iloc[test_index]
    y_train, y_test = y_data_label[train_index], y_data_label[test_index]

    model = RandomForestClassifier(random_state=42, **best_params)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Fold {fold} - Akurasi: {accuracy}")

    # Save the model to the folder
    model_filename = os.path.join(folder_name, f"model_fold_{fold}.joblib")
    joblib.dump(model, model_filename)

In [4]:
import os
import joblib
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from matplotlib.colors import LinearSegmentedColormap

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define parameter grid for Grid Search
param_grid = {
    'n_estimators': [250],
    'max_depth': [4],  # XGBoost uses max_depth instead of max_depth
    'min_child_weight': [3],  # Similar to min_samples_split in RandomForest
    'gamma': [0.3],  # Regularization parameter
    'subsample': [0.3],  # Fraction of samples used for fitting trees
    'colsample_bytree': [1],  # Fraction of features used for fitting trees
    'learning_rate': [0.3]  # Step size shrinkage used to prevent overfitting
}

# Create a GridSearchCV instance
grid = GridSearchCV(estimator=xgb.XGBClassifier(random_state=42), param_grid=param_grid, cv=skf, n_jobs=-1)

# Perform Grid Search
grid.fit(X_data_bfr, y_data_label)

# Get the best hyperparameters
best_params = grid.best_params_

# Create a folder name based on the best parameters
folder_name = f"E:/backup_CML_1/New_Drugs/Li_Endo/Result_Revision_2/XGB_8_feature_nestimators_{best_params['n_estimators']}_maxdepth_{best_params['max_depth']}_minchild_{best_params['min_child_weight']}_gamma_{best_params['gamma']}_subsample_{best_params['subsample']}_colsample_{best_params['colsample_bytree']}_lr_{best_params['learning_rate']}"
os.makedirs(folder_name, exist_ok=True)

# Train the best model using 10-fold cross-validation and save models
fold = 0

for train_index, test_index in skf.split(X_data_bfr, y_data_label):
    fold += 1
    X_train, X_test = X_data_bfr.iloc[train_index], X_data_bfr.iloc[test_index]
    y_train, y_test = y_data_label[train_index], y_data_label[test_index]

    model = xgb.XGBClassifier(random_state=42, **best_params)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Fold {fold} - Akurasi: {accuracy}")

    # Save the model to the folder
    model_filename = os.path.join(folder_name, f"model_fold_{fold}.joblib")
    joblib.dump(model, model_filename)


Fold 1 - Akurasi: 0.9840909090909091
Fold 2 - Akurasi: 0.9845454545454545
Fold 3 - Akurasi: 0.9852272727272727
Fold 4 - Akurasi: 0.9843181818181819
Fold 5 - Akurasi: 0.9872727272727273


In [ ]:
data_testA  = pd.read_csv("F:/7. Result/running_bu_yunen/duta_testing_ORD.csv")  # Ganti dengan lokasi dataset A
data_testA1 = data_testA.drop(columns=['Sample_ID', 'Drug', 'risk_level'])
print(data_testA1)

In [ ]:
X_data_test_bfr = data_testA1.drop(['risk_code'], axis=1)
y_data_test_lbl = data_testA1['risk_code']

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier

# Load the Random Forest model
model_dir = "F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/"

# List all model files in the directory
model_files = glob.glob(os.path.join(model_dir, '*.joblib'))

# Initialize a list to store the results
results = []

# Loop through each model
for model_file in model_files:
    # Load the Random Forest model
    model = joblib.load(model_file)

    # Make predictions on the test data
    y_pred = model.predict(X_data_test_bfr)

    # Calculate evaluation metrics (e.g., accuracy and F1 score)
    accuracy = accuracy_score(y_data_test_lbl, y_pred)
    f1 = f1_score(y_true=y_data_test_lbl, y_pred=y_pred, average='weighted')  # Or 'micro', 'macro', etc., as needed

    # Save the test results
    result = {
        'Model': os.path.basename(model_file),
        'Accuracy': accuracy,
        'F1 Score': f1
    }
    results.append(result)

# Create a dataframe from the test results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/Hasil_one_test_ORd_01.csv', index=False)


In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support
from xgboost import XGBClassifier
import xgboost as xgb
import joblib
from sklearn.metrics import fbeta_score, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

model_test = joblib.load('F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/model_fold_2.joblib')
# model_test = load_model("F:/7. Result/Chantest_model/save_model_02-node1_12-node2_8-oprimizer_rmsprop/Model-20231017-135318-fold-4-epoch-3.hdf5")

In [ ]:
explainer = shap.TreeExplainer(model_test)
shap_values = explainer.shap_values(X_data_test_bfr)

In [ ]:
shap.summary_plot(shap_values, plot_type="bar", feature_names = X_train.columns, show=False)
plt.ylabel("Features")
plt.xlabel("Mean (|SHAP Value|)")
plt.savefig("F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/XGB_Sum_Feature_Importance.jpg")
# plt.savefig("my_dependence_plot.pdf") # we can save a PDF of the figure if we want
plt.show()

In [ ]:
shap.summary_plot(shap_values, plot_type="bar", class_names= ['High-risk', 'Intermediate-risk', 'Low-risk'], feature_names = X_train.columns, show=False)
plt.ylabel("Features")
plt.xlabel("Mean (|SHAP Value|)")
plt.savefig("F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/XGB_Sum_Feature_Importance_label.jpg")
# plt.savefig("my_dependence_plot.pdf") # we can save a PDF of the figure if we want
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Daftar warna yang akan digunakan, satu warna untuk setiap kelas
class_colors = ['#077CDB', '#EC2CAC', '#7E8511']

# Output directory for saving TIFF files
output_directory = 'F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterasi untuk setiap kelas
for class_index in range(len(shap_values)):
    j = str(class_index)
    
    # Menghitung expected value secara keseluruhan
    overall_expected_value = explainer.expected_value[0]

    # Menggabungkan SHAP values dari beberapa baris data (tanpa nilai absolut)
    merged_shap_values = np.mean(np.abs(shap_values[class_index]), axis=0)

    scaled_shap_values = merged_shap_values

    # Memilih warna berdasarkan indeks kelas
    class_color = class_colors[class_index % len(class_colors)]

    plt.figure(figsize=(10, 6))
    bars = plt.barh(X_data_test_bfr.columns, scaled_shap_values, color=class_color)
    plt.xlabel('SHAP Values')
    plt.ylabel('Feature Names')
    plt.title("SHAP Values for Class_" + j)
    plt.grid(axis='x')
    # Menambahkan nilai detail pada setiap batang
    # Menambahkan nilai detail pada setiap batang di luar batang, di paling kanan
    for bar, shap_value in zip(bars, scaled_shap_values):
        plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{shap_value:.3f}', ha='left', va='center', color='black')

    # Save the plot as a TIFF file with DPI 300
    output_file_path = os.path.join(output_directory, f'SHAP_Plot_Class_{j}.tif')
    plt.savefig(output_file_path, format='tiff', dpi=300)
    plt.close()  # Close the plot to avoid displaying it in the notebook

print("Plots saved successfully.")

In [ ]:
import matplotlib.pyplot as plt

# Daftar warna yang akan digunakan, satu warna untuk setiap kelas
class_colors = ['#7E8511','#077CDB', '#EC2CAC']

# Iterasi untuk setiap kelas
for class_index in range(len(shap_values)):
    j = str(class_index)
    
    # Menghitung expected value secara keseluruhan
    overall_expected_value = explainer.expected_value[0]

    # Menggabungkan SHAP values dari beberapa baris data (tanpa nilai absolut)
    merged_shap_values = np.mean(np.abs(shap_values[class_index]), axis=0)

    scaled_shap_values = merged_shap_values

    # Memilih warna berdasarkan indeks kelas
    class_color = class_colors[class_index % len(class_colors)]

    plt.figure(figsize=(10, 6))
    bars = plt.barh(X_data_test_bfr.columns, scaled_shap_values, color=class_color)
    plt.xlabel('SHAP Values')
    plt.ylabel('Feature Names')
    plt.title("SHAP Values for Class_" + j)
    plt.grid(axis='x')
    # Menambahkan nilai detail pada setiap batang
    # Menambahkan nilai detail pada setiap batang di luar batang, di paling kanan
    for bar, shap_value in zip(bars, scaled_shap_values):
        plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{shap_value:.3f}', ha='left', va='center', color='black')
    plt.show()

In [ ]:
import shap

for i in range(len(shap_values)):
    # Menghitung expected value secara keseluruhan
    overall_expected_value = explainer.expected_value[0]

    # Menggabungkan SHAP values dari beberapa baris data
    merged_shap_values = shap_values[i].mean(axis=0)

    scaled_shap_values = merged_shap_values * 10
    
    plt.figure(figsize=(10, 6))

    # Meringkas waterfall plot untuk semua baris data
    shap.plots._waterfall.waterfall_legacy(overall_expected_value, scaled_shap_values, 
                                           feature_names = X_data_test_bfr.columns, max_display=20, show=False)
    plt.savefig(output_directory + "Waterfall_HIL_Risk_{0}.jpg".format(i))
    # plt.savefig("my_dependence_plot.pdf") # we can save a PDF of the figure if we want
    plt.show()

In [ ]:
data_testAs  = pd.read_csv("F:/7. Result/running_bu_yunen/duta_testing_ORD.csv")  # Ganti dengan lokasi dataset A
data_testAs1 = data_testAs.drop(columns=['Sample_ID', 'Drug', 'risk_level'])
data_name = data_testAs['Drug']
data_test_ID = data_testAs['Sample_ID'].astype(int)

data_test_n = pd.concat([data_test_ID, data_testAs1, data_name], axis=1)

print(data_test_n)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score, roc_curve, auc  # Import roc_curve and auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib  # Import modul joblib

# Load the Random Forest model using joblib
model = joblib.load('F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/model_fold_2.joblib')  # Ganti dengan path yang sesuai

# Daftar unik dari jenis 'Drug_Name' dalam dataset
unique_drug_names = data_test_n['Drug'].unique()

# Inisialisasi list untuk menyimpan semua confusion matrix
all_confusion_matrices = []
drug_combination = []
d_X_test = []
accu = []
accu_h = []
accu_i = []
accu_l = []
f1_sc = []
f1_sc_h = []
f1_sc_i = []
f1_sc_l = []
lr_p_h = []
lr_p_i = []
lr_p_l = []
lr_n_h = []
lr_n_i = []
lr_n_l = []
auc_high = []
auc_inter = []
auc_low = []
rec_h = []
rec_i = []
rec_l = []
spe_h = []
spe_i = []
spe_l = []

# Lakukan iterasi sebanyak 10.000 kali
for _ in range(10000):
    # Pilih secara acak 16 'Drug_Name' yang berbeda
    unique_drug_names = np.random.choice(data_test_n['Drug'].unique(), 16, replace=False)

    # Inisialisasi list untuk menyimpan 16 sampel
    selected_combinations = []

    # Memilih satu sampel dengan 'Drug_Name' yang sesuai untuk setiap 'Drug_Name' yang telah dipilih
    for drug_name in unique_drug_names:
        selected_sample = data_test_n[data_test_n['Drug'] == drug_name].sample(1)
        selected_combinations.append(selected_sample)

    # Menggabungkan 16 sampel menjadi satu DataFrame
    selected_combinations = pd.concat(selected_combinations)

    # Pisahkan fitur dan target sesuai kebutuhan Anda
    X_test = selected_combinations.drop(columns=['Sample_ID', 'risk_code', 'Drug'])  # Sesuaikan dengan struktur dataset Anda
    y_test = selected_combinations['risk_code']  # Sesuaikan dengan struktur dataset Anda
#     y_test = label_encoder.fit_transform(y_test)
    
    # Uji model dengan data uji
    y_pred = model.predict_proba(X_test)   # Menggunakan model Random Forest yang telah dimuat

    # Uji model dengan data uji
    y_pred1 = model.predict(X_test)
    
    # Hitung confusion matrix
    cm = confusion_matrix(y_test, y_pred1)  # Sesuaikan dengan nilai threshold yang sesuai
    
    tp_high = cm[0,0]
    tn_high = cm[1,1]+cm[1,2]+cm[2,1]+cm[2,2]
    fp_high = cm[1,0]+cm[2,0]
    fn_high = cm[0,1]+cm[0,2]

    tp_inter = cm[1,1]
    tn_inter = cm[0,0]+cm[0,2]+cm[2,0]+cm[2,2]
    fp_inter = cm[0,1]+cm[2,1]
    fn_inter = cm[1,0]+cm[1,2]

    tp_low = cm[2,2]
    tn_low = cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1]
    fp_low = cm[0,2]+cm[1,2]
    fn_low = cm[2,0]+cm[2,1]

    acc_high = (tp_high+tn_high)/(tp_high+tn_high+fp_high+fn_high)
    pre_high = tp_high/(tp_high+fp_high)
    rec_high = tp_high/(tp_high+fn_high)
    spe_high = tn_high/(tn_high+fp_high)
    lrp_high = rec_high/(1-spe_high)
    lrn_high = (1-rec_high)/spe_high
    f1s_high = (2*pre_high*rec_high)/(pre_high+rec_high)

    acc_inter = (tp_inter+tn_inter)/(tp_inter+tn_inter+fp_inter+fn_inter)
    pre_inter = tp_inter/(tp_inter+fp_inter)
    rec_inter = tp_inter/(tp_inter+fn_inter)
    spe_inter = tn_inter/(tn_inter+fp_inter)
    lrp_inter = rec_inter/(1-spe_inter)
    lrn_inter = (1-rec_inter)/spe_inter
    f1s_inter = (2*pre_inter*rec_inter)/(pre_inter+rec_inter)

    acc_low = (tp_low+tn_low)/(tp_low+tn_low+fp_low+fn_low)
    pre_low = tp_low/(tp_low+fp_low)
    rec_low = tp_low/(tp_low+fn_low)
    spe_low = tn_low/(tn_low+fp_low)
    lrp_low = rec_low/(1-spe_low)
    lrn_low = (1-rec_low)/spe_low
    f1s_low = (2*pre_low*rec_low)/(pre_low+rec_low)
    
    acc = (acc_high+acc_inter+acc_low)/3
    f1_s = (f1s_high+f1s_inter+f1s_low)/3
    
# Hitung AUC untuk masing-masing kelas
    auc_high_value = roc_auc_score(y_test == 0, y_pred[:, 0])
    auc_inter_value = roc_auc_score(y_test == 1, y_pred[:, 1])
    auc_low_value = roc_auc_score(y_test == 2, y_pred[:, 2])

    auc_high.append(auc_high_value)
    auc_inter.append(auc_inter_value)
    auc_low.append(auc_low_value)


    # Simpan confusion matrix ke dalam list
    drug_combination.append(selected_combinations)
    all_confusion_matrices.append(cm)
    d_X_test.append(X_test)
    accu.append(acc)
    accu_h.append(acc_high)
    accu_i.append(acc_inter)
    accu_l.append(acc_low)
    f1_sc.append(f1_s)
    f1_sc_h.append(f1s_high)
    f1_sc_i.append(f1s_inter)
    f1_sc_l.append(f1s_low)
    rec_h.append(rec_high)
    rec_i.append(rec_inter)
    rec_l.append(rec_low)
    spe_h.append(spe_high)
    spe_i.append(spe_inter)
    spe_l.append(spe_low)
    lr_p_h.append(lrp_high)
    lr_p_i.append(lrp_inter)
    lr_p_l.append(lrp_low)
    lr_n_h.append(lrn_high)
    lr_n_i.append(lrn_inter)
    lr_n_l.append(lrn_low)

# Buat DataFrame dari list
df_result = pd.DataFrame({'Drug_Combination': drug_combination, 
                        'X_data': d_X_test, 
                        'Confusion_Matrix': all_confusion_matrices,
                        'Acc' : accu,
                        'Acc_High': accu_h,
                         'Acc_Inter': accu_i,
                         'Acc_Low': accu_l,
                        'AUC_High': auc_high,
                        'AUC_Inter': auc_inter,
                        'AUC_Low': auc_low,
                          'Sens_High': rec_h,
                          'Sens_Inter': rec_i,
                          'Sens_Low': rec_l,
                          'Spec_High': spe_h,
                          'Spec_Inter': spe_i,
                          'Spec_Low': spe_l,
                         'F1_S': f1_sc,
                         'F1_S_High': f1_sc_h,
                         'F1_S_Inter': f1_sc_i,
                         'F1_S_Low': f1_sc_l,
                         'LR_p_High': lr_p_h,
                         'LR_p_Inter': lr_p_i,
                         'LR_p_Low': lr_p_l,
                         'LR_n_High': lr_n_h,
                         'LR_n_Inter': lr_n_i,
                         'LR_n_Low': lr_n_l})

df_result.to_csv('F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/Result_of_10000_Times_Test_of_RF_01.csv')

In [ ]:
import matplotlib.pyplot as plt
# Define custom colors and edge colors
color_high = '#7E8511'  # Custom color for AUC_High
color_inter = '#077CDB'  # Custom color for AUC_Inter
color_low = '#EC2CAC'  # Custom color for AUC_Low
edge_color = 'black'  # Edge color

# Create a figure for the histograms
plt.figure(figsize=(12, 6))

# Histogram for 'AUC_High'
plt.subplot(131)
plt.hist(auc_high, bins=20, color=color_high, edgecolor=edge_color)
plt.title('AUC_High Histogram')
plt.xlabel('AUC_High')
plt.ylabel('Frequency')

# Histogram for 'AUC_Inter'
plt.subplot(132)
plt.hist(auc_inter, bins=20, color=color_inter, edgecolor=edge_color)
plt.title('AUC_Inter Histogram')
plt.xlabel('AUC_Inter')
plt.ylabel('Frequency')

# Histogram for 'AUC_Low'
plt.subplot(133)
plt.hist(auc_low, bins=20, color=color_low, edgecolor=edge_color)
plt.title('AUC_Low Histogram')
plt.xlabel('AUC_Low')
plt.ylabel('Frequency')

plt.tight_layout()  # Ensure the subplots don't overlap

# Define the file path where you want to save the histograms
output_file_path = 'F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/AUC.png'

# Save the figure to the specified file
plt.savefig(output_file_path, dpi=300, bbox_inches='tight')

# Show the plots (optional)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Assuming df_result is your DataFrame
columns_of_interest = ['Acc', 'Acc_High', 'Acc_Inter', 'Acc_Low', 'AUC_High', 'AUC_Inter', 'AUC_Low', 'F1_S', 'F1_S_High', 'F1_S_Inter', 'F1_S_Low', 'LR_p_High', 'LR_p_Inter', 'LR_p_Low', 'LR_n_High', 'LR_n_Inter', 'LR_n_Low']

# Calculate the median and confidence interval
medians = df_result[columns_of_interest].median()
lower_bound = df_result[columns_of_interest].quantile(0.025)
upper_bound = df_result[columns_of_interest].quantile(0.975)

# Combine median, lower, and upper into a single string with two decimal places
result_strings = medians.round(2).astype(str) + ' (' + lower_bound.round(2).astype(str) + ', ' + upper_bound.round(2).astype(str) + ')'

# Create a new DataFrame with the desired format
table = pd.DataFrame({'Median (Lower_CI, Upper_CI)': result_strings})

# Add a new column for the metric names
table['auc_detail'] = columns_of_interest

# Reorder the columns
table = table[['auc_detail', 'Median (Lower_CI, Upper_CI)']]
# Save the table to Excel
table.to_excel('F:/7. Result/running_bu_yunen/ord/XGB_14_feature_nestimators_500_maxdepth_6_minchild_4_gamma_0_subsample_0.8_colsample_1.0_lr_0.3/Final_result_RF_01.xlsx', index=False)

# Display the table
table


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats  # Add this import for the stats module

# Function to calculate 95% CIs
def calculate_ci(data):
    mean_val = np.mean(data)
    ci_low, ci_high = stats.t.interval(0.95, len(data)-1, loc=mean_val, scale=stats.sem(data))
    return ci_low, ci_high

In [ ]:
# Columns to calculate statistics for
columns_of_interest = ['Acc', 'Acc_High', 'Acc_Inter', 'Acc_Low', 'AUC_High', 'AUC_Inter', 'AUC_Low', 'Sens_High', 'Sens_Inter','Sens_Low','Spec_High','Spec_Inter','Spec_Low','F1_S', 'F1_S_High', 'F1_S_Inter', 'F1_S_Low', 'LR_p_High', 'LR_p_Inter', 'LR_p_Low', 'LR_n_High', 'LR_n_Inter', 'LR_n_Low']

# Calculate median and CIs
table = df_result[columns_of_interest].agg(['median', calculate_ci]).T.reset_index()
# print(table)
table.columns = ['metric', 'median', 'ci']  # Corrected column names
print(table)

# Save to Excel
table.to_excel('D:/backup_CML_1/New_Drugs/Li_Endo/RandomForest_model_03_nestimators_100_maxdepth_None_minsplit_3_minleaf_1/Final_result_RF_01.xlsx', index=False)

In [ ]:
table = df_result[['Acc', 'Acc_High','Acc_Inter','Acc_Low', 'AUC_High', 'AUC_Inter', 'AUC_Low','F1_S','F1_S_High','F1_S_Inter','F1_S_Low','LR_p_High','LR_p_Inter','LR_p_Low', 'LR_n_High', 'LR_n_Inter','LR_n_Low']].agg(['mean', 'median', 'min', 'max']).apply(lambda x: round(x, 2)).reset_index().rename(columns={'index':'auc_detail'})

table.to_excel('D:/backup_CML_1/New_Drugs/Li_Endo/RandomForest_model_02_nestimators_300_maxdepth_None_minsplit_5_minleaf_1/Final_result_RF_01.xlsx', index = False)

table